In [ ]:
from IPython.display import clear_output

!pip install datasets transformers rouge_score nltk

# !pip install -q sentencepiece
# !pip install rouge-score # google package version

clear_output()

In [ ]:
import os
import re
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

# pytorch dataset types
import datasets
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, load_metric

import tensorflow as tf
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration # pegasus
from transformers import BartTokenizer, TFBartForConditionalGeneration # bart
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
data_dir = "/content/gdrive/MyDrive/Classes/W266_NLP/w266_reddit_summarization/data/tensorflow_datasets/reddit/1.0.0"
os.chdir(data_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# define train/test split from the data. but don't read the data yet.
file_names = os.listdir(data_dir)
metadata_files = [i for i in file_names if re.search('^(?!reddit-train)', i)]

data_files = np.array([i for i in file_names if re.search('^reddit-train', i)])
split_ind = np.array([int(x[22:27]) % 5 for x in data_files])
test = data_files[split_ind == 0]
train = data_files[split_ind != 0]

In [ ]:
# shorten the data for now, to make everything simpler
valid = train[4:5]
train = train[:3]
test = test[:2]

In [ ]:
start = time.time()
def parse_file(serialized_example, return_xy=False):

  file_scruct = {
      'author': tf.io.FixedLenFeature([], tf.string),
      'body': tf.io.FixedLenFeature([], tf.string),
      'content': tf.io.FixedLenFeature([], tf.string),
      'id': tf.io.FixedLenFeature([], tf.string),
      'normalizedBody': tf.io.FixedLenFeature([], tf.string),
      'subreddit': tf.io.FixedLenFeature([], tf.string),
      'subreddit_id': tf.io.FixedLenFeature([], tf.string),
      'summary': tf.io.FixedLenFeature([], tf.string),
  }

  example_parsed = tf.io.parse_single_example(serialized_example, file_scruct)
  return example_parsed


# parse
train_tf = tf.data.TFRecordDataset(train).map(parse_file)
valid_tf = tf.data.TFRecordDataset(valid).map(parse_file)
test_tf = tf.data.TFRecordDataset(test).map(parse_file)

decode_string = np.vectorize(lambda x: x.decode('utf-8'))

# load data into memory into dictionary. 
# figure out how to bypass this
## we're doing tf -> np -> pt. Want to go tf -> pt
def tf_to_dict(tf_item):
  dataset_dict = {'content': [], 'summary': [], 'subreddit': []}
  for item in tf_item: 
    dataset_dict['content'].append(str(decode_string(item['content'].numpy())))
    dataset_dict['summary'].append(str(decode_string(item['summary'].numpy())))
    dataset_dict['subreddit'].append(str(decode_string(item['subreddit'].numpy())))
  return dataset_dict
# def tf_to_dict(tf_item):
#   dataset_dict = {'content': [], 'summary': [], 'subreddit': []}
#   for item in tf_item: 
#     dataset_dict['content'].append(item['content'])
#     dataset_dict['summary'].append(item['summary'])
#     dataset_dict['subreddit'].append(item['subreddit'])
#   return dataset_dict

# convert to pytorch Dataset datatype
all_data = DatasetDict({
    'train': Dataset.from_dict(tf_to_dict(train_tf)), 
    'valid': Dataset.from_dict(tf_to_dict(valid_tf)), 
    'test': Dataset.from_dict(tf_to_dict(test_tf))})

seconds_elapsed = time.time() - start
print(f"{seconds_elapsed/60} minutes elapsed")

1.1519737203915914 minutes elapsed


In [ ]:
# visualize some examples
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


show_random_elements(all_data['train'])

# Load BART

In [ ]:
from transformers import BartTokenizer, TFBartForConditionalGeneration
checkpoint = 'facebook/bart-base' # "facebook/bart-large"
# "t5-small" # try t5 small later
model = TFBartForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = BartTokenizer.from_pretrained(checkpoint)

# load the metric we'll use which is rouge
metric = load_metric('rouge')

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
# one example, no fine tuning
inputs = tokenizer(all_data['train']['content'][0], max_length=1024, truncation=True, return_tensors="tf", padding='max_length')
summary_ids = model.generate(inputs["input_ids"], 
                              num_beams=1,
                              no_repeat_ngram_size=1,
                              min_length=1,
                              max_length=150)
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False), compact=True)

['Here is something Olympic middle distance runner Nick Willis wrote about '
 'fellow Kiwi Sean Adams (Pitt) Â In the history of basketball in New Zealand, '
 'it has been a long time since there have ever had two players reach this '
 'level. The first was Steven Marks from Portland State University who went '
 'un-drafted and never played for an NBA team over his 11 season career with '
 'them until he made enough to be drafted by Boston College as well on their '
 'roster during that same year’s draft class at age 19 years old:Sean marks '
 'also reached great heights when playing college ball under coach Jamie '
 'Dixon; Mark Davis Jr., another former Oregon native came out victorious '
 'against Detroit Pistons after only one game while still being selected #1 '
 'overall pick']


In [ ]:
# tokenize summary and content
max_input_length = 1024
max_target_length = 128

model_checkpoint = checkpoint

# if we're using t5, append "summarize: " in front
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

def preprocess_function(examples, xvar='content', yvar='summary', max_input_length=1024, max_target_length=128):
    inputs = [prefix + doc for doc in examples[xvar]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples[yvar], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# now tokenize our data
all_data_tokenized2 = all_data.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f3a1d740320> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/31 [00:00<?, ?ba/s]

In [ ]:
# inspect. so now the input_ids are tokenized inputs. and the labels are tokenized summaries. 
all_data_tokenized2['train']

Dataset({
    features: ['content', 'summary', 'subreddit', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 45097
})

In [ ]:
# set params
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

# use collator to convert our PT dataset to TF dataset
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

# NOTE, I'M TRUNCATING THE DATA TO ONLY 10 OBS TO SPEED THIS UP
# REMOVE THE .select() LATER
train_dataset = all_data_tokenized2["train"].select(range(10)).to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = all_data_tokenized2["valid"].select(range(10)).to_tf_dataset(
    batch_size=8,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
# model.compile(optimizer=optimizer)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

2/2 [==============================] - 34s 5s/step - loss: nan - sparse_categorical_accuracy: 0.1015 - val_loss: nan - val_sparse_categorical_accuracy: 0.0078


In [ ]:
print("Input post:")
pprint(all_data_tokenized2["valid"]['content'][0])
print("\ndetokenizing")
pprint(tokenizer.batch_decode(all_data_tokenized2['valid']['input_ids'][0], 
                              skip_special_tokens=True, clean_up_tokenization_spaces=False), compact=True)

Input post:
('Since I am blessed with the ability to use /gamemode, I plan to be in '
 'survival as much as possible when 0.9.0 comes out. I tried with this current '
 'world, but the temptation was too great!  Personally, I like to challenge '
 'myself with odd and gigantic builds. It helps keep me focused, and (I hope ) '
 'serves as an example of what could be done by other players who still think '
 'in terms of rectangular structures. Up until now, creative has been the '
 'fastest and most efficient way to do this, while still being able to police '
 'the world and help others with my limited playing time.')

detokenizing
['<s>', 'Since', ' I', ' am', ' blessed', ' with', ' the', ' ability', ' to',
 ' use', ' /', 'gam', 'em', 'ode', ',', ' I', ' plan', ' to', ' be', ' in',
 ' survival', ' as', ' much', ' as', ' possible', ' when', ' 0', '.', '9', '.',
 '0', ' comes', ' out', '.', ' I', ' tried', ' with', ' this', ' current',
 ' world', ',', ' but', ' the', ' temptation', ' was', 

Compiled model not working...

In [ ]:
# make prediction:
# maybe figure out how to pass in the pre encoded input: all_data_tokenized2
inputs = tokenizer(all_data['valid']['content'][0], max_length=1024, truncation=True, return_tensors="tf", padding='max_length')
summary_ids = model.generate(inputs, 
                              num_beams=1,
                              no_repeat_ngram_size=1,
                              min_length=1,
                              max_length=150)
print("Predicted summary:")
pprint(tokenizer.batch_decode(summary_ids, 
                              skip_special_tokens=True, clean_up_tokenization_spaces=False), compact=True)

print("\nTrue summary:")
pprint(all_data_tokenized2["valid"]['summary'][0])

pprint("\nTrue summary decoded:")
pprint(tokenizer.batch_decode(all_data_tokenized2['valid']['labels'][0], 
                              skip_special_tokens=True, clean_up_tokenization_spaces=False), compact=True)

AttributeError: ignored

In [ ]:
summary_ids = model.generate(validation_dataset, 
                              num_beams=1,
                              no_repeat_ngram_size=1,
                              min_length=1,
                              max_length=150)


AttributeError: ignored

# Try pytorch

In [ ]:
# we already have our tokenized dataset to use here: all_data_tokenized2
# restart model:
checkpoint = 'facebook/bart-base' 
#model = TFBartForConditionalGeneration.from_pretrained(checkpoint)
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [ ]:
# set hyperparms
args = Seq2SeqTrainingArguments(
    output_dir = 'finetuned_bart_v1', # file path to save checkpoints of the model. 
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3, # only make 3 checkpoint saves maximum through training process. 
    num_train_epochs=1,
    predict_with_generate=True, # use this to predict summaries. 
    fp16=True, # default is false, whether to use 16-bit precision training instead of 32
    # push_to_hub=True,
)

In [ ]:
# define how to compute metrics from preds. 
# We'll use rouge from the 'metric' object we defined above
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# need this collator to pad the examples
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=all_data_tokenized2["train"],
    eval_dataset=all_data_tokenized2["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

RuntimeError: ignored